In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import os, warnings

In [4]:
warnings.simplefilter('ignore')

In [5]:
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from ast import literal_eval

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\utkarsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\utkarsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\utkarsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\utkarsh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\utkarsh\AppData\Roaming\nltk

In [6]:
from nltk.corpus import stopwords

In [7]:
from nltk.tokenize import word_tokenize

In [8]:
from ast import literal_eval

In [9]:
df = pd.read_csv('restaurants.csv')

In [11]:
df['City'].nunique()

23

In [12]:
df.drop_duplicates(inplace=True)

In [13]:
df.shape

(6590, 8)

In [14]:
df.reset_index(drop = True, inplace = True)

# Processing Location Columns

In [15]:
def extract_area(loc):
    splits = loc.split(',')
    if len(splits) == 2:
        splits.pop(1)
    elif len(splits) == 3:
        splits.pop(2)
    elif len(splits) == 4:
        splits.pop(3)
    final_loc = ' '.join(l for l in splits)
    return final_loc

df['Location'] = df['Location'].apply(lambda x: extract_area(x))

# Preprocessing Cuisine And City

In [16]:
def preprocess(cuisine):
    cuisine = cuisine.lower()
    cuisine = cuisine.replace(',',' ')
    return cuisine

In [17]:
df['Cuisine'] = df['Cuisine'].apply(lambda x: preprocess(x))
df['City'] = df['City'].apply(lambda x: preprocess(x))

In [18]:
unwanted_cols = ['Locality','Votes']
df.drop(unwanted_cols,axis=1,inplace=True)

In [19]:
df.head()

,Name,Location,City,Cuisine,Rating,Cost
0,Local,Scindia House Connaught Place,delhi,north indian finger food continental,4.1,2000
1,The G.T. ROAD,M-Block Connaught Place,delhi,north indian,4.3,1500
2,Tamasha,Connaught Place,delhi,finger food north indian italian contine...,4.2,2000
3,The Junkyard Cafe,Connaught Place,delhi,north indian mediterranean asian italian...,4.2,1800
4,Chili's American Grill and Bar,M-Block Connaught Place,delhi,mexican american italian,4.4,2000


# Recommend me a Restraunt

In [20]:


def recommend_restaurant(city, cuisine, cost):
    cuisine = cuisine.lower()

    # Filtering the cuisine set
    stop_words = set(stopwords.words('english'))
    filtered = {word for word in word_tokenize(cuisine) if word not in stop_words}

    # Filtering the dataset based on input city and cost
    city_df = df[df['City'].str.lower() == city.lower()]
    city_df = city_df[city_df['Cost'] <= int(cost)]
    city_df = city_df.set_index(np.arange(city_df.shape[0]))

    cos = []
    for i in range(city_df.shape[0]):
        temp_token = word_tokenize(city_df["Cuisine"][i])
        temp_set = {word for word in temp_token if word not in stop_words}
        vector = temp_set.intersection(filtered)
        cos.append(len(vector))
    
    # Assigning similarity scores
    city_df['similarity'] = cos

    # Sorting based on similarity and rating
    city_df = city_df.sort_values(by=['similarity', 'Rating'], ascending=[False, False])
    city_df.reset_index(drop=True, inplace=True)
    
    return city_df[["Name", "Location", "Rating"]].head()


In [21]:
recommend_restaurant("Delhi","Dessert",500)

,Name,Location,Rating
0,Bistro 55,Paharganj,5.0
1,The Waffle Co.,Rajinder Nagar,5.0
2,Bake Club,East Patel Nagar,5.0
3,3 Friends Pizza,Pandav Nagar,5.0
4,Cafe Emoji's,Moti Bagh,5.0


In [22]:
ans = recommend_restaurant('Mumbai','Desserts',2000)
ans

,Name,Location,Rating
0,Elementaria Bakery Cafe,Powai,5.0
1,Wowfillss,Parel,5.0
2,The Pancake Story,Narayan Niwas Vile Parle East,5.0
3,99 Pancakes,Silver Croft Building Bandra West,5.0
4,Frozen Bottle,Hiranandani,4.7


In [23]:
df.head()

,Name,Location,City,Cuisine,Rating,Cost
0,Local,Scindia House Connaught Place,delhi,north indian finger food continental,4.1,2000
1,The G.T. ROAD,M-Block Connaught Place,delhi,north indian,4.3,1500
2,Tamasha,Connaught Place,delhi,finger food north indian italian contine...,4.2,2000
3,The Junkyard Cafe,Connaught Place,delhi,north indian mediterranean asian italian...,4.2,1800
4,Chili's American Grill and Bar,M-Block Connaught Place,delhi,mexican american italian,4.4,2000


In [24]:
import pickle

In [25]:
pickle.dump(df, open('hotel.pkl', 'wb'))